In [1]:
import numpy as np
from scipy.integrate import quad
from astropy import units as u
from astropy.constants import e, m_e, m_p, c, N_A

# Constants and initial parameters
E_0 = 76e9 * u.eV  # Initial energy in eV (76 GeV for U-70 Synchrotron protons)
L = 10 * u.cm  # Thickness of the material in cm (e.g., water or tissue)
I = 1e-6 * u.A  # Estimated current in Amperes (1 microampere)
T = 3 * u.s  # Estimated exposure time

# Material properties for water (H2O)
Z_eff = 7.42  # Effective atomic number for water
A_eff = 18.015 * u.g / u.mol  # Molar mass of water in g/mol

# Define mean excitation potential (I) for water (approximate value)
I_water = (75 * u.eV).to(u.J)  # Mean excitation potential in Joules

# Define the constant k (in MeV cm^2/g), converted to SI units
k = (0.307075 * u.MeV * u.cm**2 / u.g).to(u.J * u.m**2 / u.kg)  # Conversion to SI units

# Convert A_eff to kg for use in the equation by dividing by Avogadro's number to get mass per atom
A_eff_per_atom = (A_eff / N_A).to(u.kg)

# Function for the Bethe-Bloch energy loss per unit distance (dE/dx) in SI units
def bethe_bloch(E, Z, A, I):
    # Convert E to Joules for consistency
    E = E.to(u.J)
    gamma = E / (m_p * c**2)  # Lorentz factor
    beta = np.sqrt(1 - (1 / gamma)**2)  # Velocity factor
    
    # Bethe-Bloch formula in SI units
    dE_dx = k * (Z / A) * (1 / beta**2) * (
        np.log((2 * m_e * c**2 * beta**2 * gamma**2 / I).decompose()) - beta**2
    )
    
    # Result in units of J/m
    return dE_dx

# Integrate the Bethe-Bloch energy loss over the material thickness L for a single proton
def total_energy_loss(E_0, L, Z, A, I):
    # Integrate over x from 0 to L
    energy_loss, _ = quad(lambda x: bethe_bloch(E_0, Z, A, I).value, 0, L.to(u.m).value)
    return energy_loss * u.J  # Result in Joules

# Calculate total energy deposited in the material over L for a single proton
E_deposit_per_proton = total_energy_loss(E_0, L, Z_eff, A_eff_per_atom, I_water)

# Calculate final energy after passing through material for one proton
E_final_single_proton = max(E_0.to(u.J) - E_deposit_per_proton, 0 * u.J)  # Ensure non-negative energy

# Relativistic calculation of velocity from energy
def relativistic_velocity(E):
    gamma = E / (m_p * c**2)
    beta = np.sqrt(1 - 1 / gamma**2)
    return (beta * c).to(u.m / u.s)

# Relativistic calculation of momentum from energy
def relativistic_momentum(E):
    return np.sqrt((E / c)**2 - (m_p * c)**2).to(u.kg * u.m / u.s)

# Initial and final velocities
v_0 = relativistic_velocity(E_0.to(u.J))  # Initial velocity (relativistic)
v_final = relativistic_velocity(E_final_single_proton) if E_final_single_proton > 0 * u.J else 0 * u.m / u.s  # Final velocity

# Initial and final momenta
p_0 = relativistic_momentum(E_0.to(u.J))  # Initial momentum
p_final = relativistic_momentum(E_final_single_proton) if E_final_single_proton > 0 * u.J else 0 * u.kg * u.m / u.s  # Final momentum

# Momentum transfer per proton
delta_p_per_proton = p_0 - p_final

# Calculate number of protons per second
protons_per_second = (I / e.si).si  # Result in protons per second

# Total momentum transfer and energy deposition over time T
total_energy_deposit = E_deposit_per_proton * protons_per_second * T  # Total energy in Joules
total_momentum_transfer = delta_p_per_proton * protons_per_second * T  # Total momentum transfer in kg*m/s

# Results
print(f"Total Energy Deposited over {L:.2f} in {T:.2f} s: {total_energy_deposit:.2e}")
print(f"Initial Momentum of a single proton: {p_0:.2e}")
print(f"Final Momentum of a single proton after passing through {L:.2f} cm: {p_final:.2e}")
print(f"Momentum Transfer per proton: {delta_p_per_proton:.2e}")
print(f"Total Momentum Transfer over {T} seconds: {total_momentum_transfer:.2e}")


Total Energy Deposited over 10.00 cm in 3.00 s s: 3.96e+25 J
Initial Momentum of a single proton: 4.06e-17 kg m / s
Final Momentum of a single proton after passing through 10.00 cm cm: 0.00e+00 kg m / s
Momentum Transfer per proton: 4.06e-17 kg m / s
Total Momentum Transfer over 3.0 s seconds: 7.60e-04 kg m / s


In [2]:
E_deposit_per_proton

<Quantity 2.11254282e+12 J>

In [3]:
E_deposit_per_proton = total_energy_loss(E_0, L, Z_eff, A_eff_per_atom, I_water)